In [ ]:
import os
import pandas as pd
from gensim import corpora, models, similarities

In [ ]:
datafile = '../input/bias-media-cat/all3.csv'

In [ ]:
import pandas as pd
tweets = pd.read_csv(datafile, encoding='utf8')
#// tweets = tweets.assign(Time=pd.to_datetime(tweets.time)).drop('tweet_id', axis='columns')
#x = ""+tweets.year+"/"+tweets.month + "/" +tweets.date +" " +tweets.time
#tweets = tweets.assign(Time=pd.to_datetime(x)).drop('tweet_id', axis='columns')

tweets.head(10)

In [ ]:
range(len(tweets['tweet']))

range( len(tweets[tweets['tweet'].str.contains("atal|Catalonia|Catalunya|Cataluña")==True]['tweet'] ))

In [ ]:
corpus=[]
a=[]
# remove punctiation
tweets["tweet"] = tweets['tweet'].str.replace('[^\w\s]','')
tweets["tweet"] = tweets["tweet"] + " " + tweets["screen_name"] 
for a in tweets[tweets['tweet'].str.contains("Catalonia|Catalunya|Cataluña")==True]['tweet']:
        corpus.append(a)

In [ ]:
corpus[0:5]

In [ ]:
import gensim
import logging
import tempfile

TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

from gensim import corpora
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [ ]:
from nltk.corpus import stopwords
from string import punctuation

# remove common words and tokenize
list1 = ['RT','rt','es','el','per','amb','pel','parter','directo','és','els','tras','minuto','hora','social']
stoplist =  stopwords.words('spanish') + stopwords.words('english') + list(punctuation) + list1

texts = [[word for word in str(document).lower().split() if word not in stoplist] for document in corpus]

In [ ]:
dictionary = corpora.Dictionary(texts)
dictionary.save(os.path.join(TEMP_FOLDER, 'elon.dict'))  # store the dictionary, for future reference
#print(dictionary)

In [ ]:
#print(dictionary.token2id)

In [ ]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize(os.path.join(TEMP_FOLDER, 'elon.mm'), corpus)  # store to disk, for later use

In the previous cells, we created a corpus of documents represented as a stream of vectors. To continue, let’s fire up gensim and use that corpus:

In [ ]:
from gensim import corpora, models, similarities

### Creating a transformation


The transformations are standard Python objects, typically initialized by means of a training corpus:

Different transformations may require different initialization parameters; in case of TfIdf, the “training” consists simply of
going through the supplied corpus once and computing document frequencies of all its features.
Training other models, such as Latent Semantic Analysis or Latent Dirichlet Allocation, is much more involved and,
consequently, takes much more time.

In [ ]:
tfidf = models.TfidfModel(corpus) # step 1 -- initialize a model

### Note
Transformations always convert between two specific vector spaces. The same vector space (= the same set of feature ids) must be used for training as well as for subsequent vector transformations. Failure to use the same input feature space, such as applying a different string preprocessing, using different feature ids, or using bag-of-words input vectors where TfIdf vectors are expected, will result in feature mismatch during transformation calls and consequently in either garbage output and/or runtime exceptions.

From now on, tfidf is treated as a read-only object that can be used to apply a transformation to a whole corpus:

In [ ]:
corpus_tfidf = tfidf[corpus]  # step 2 -- use the model to transform vectors

### LDA:
https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation

Latent Dirichlet Allocation, LDA is yet another transformation from bag-of-words counts into a topic space of lower dimensionality. LDA is a probabilistic extension of LSA (also called multinomial PCA), so LDA’s topics can be interpreted as probability distributions over words. These distributions are, just like with LSA, inferred automatically from a training corpus. Documents are in turn interpreted as a (soft) mixture of these topics (again, just like with LSA).

In [ ]:
total_topics = 5

In [ ]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=total_topics)
corpus_lda = lda[corpus_tfidf] # create a double wrapper over the original corpus: bow->tfidf->fold-in-lsi

In [ ]:
#Show first n important word in the topics:
lda.show_topics(total_topics,5)

In [ ]:
from collections import OrderedDict

data_lda = {i: OrderedDict(lda.show_topic(i,25)) for i in range(total_topics)}
#data_lda

In [ ]:
import pandas as pd

df_lda = pd.DataFrame(data_lda)
print(df_lda.shape)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)

In [ ]:
df_lda

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

g=sns.clustermap(df_lda.corr(), center=0, cmap="RdBu", metric='cosine', linewidths=.75, figsize=(12, 12))
plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
plt.show()
#plt.setp(ax_heatmap.get_yticklabels(), rotation=0)  # For y axis

In [ ]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda, corpus_lda, dictionary, mds='tsne')
panel